# View LSSTComCam DeepCoadd On selected Strong Lens in ECDFS

- author Sylvie Dagoret-Campagne
- creation date 2025-05-10
- last update 2025-05-10
- last update 2025-05-10
- last verification: 2025-05-27
- LSST pipelines : w_2025_17

## Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

# import lsst.daf.butler as dafButler
from lsst.daf.butler import Butler

import lsst.geom as geom
from lsst.geom import SpherePoint, degrees
import lsst.afw.display as afwDisplay

from lsst.skymap import PatchInfo, Index2D

In [ ]:
# For Angle conversion
from astropy.coordinates import Angle
import astropy.units as u

In [ ]:
all_bands = ["u", "g", "r", "i", "z", "y"]
all_bands_colors = ["blue", "green", "red", "orange", "yellow", "purple"]

## Config

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = "/repo/main"
# collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666' # work
# collection = 'LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810' # work
collection = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"  # work


# bad : crash collection = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'

# bad : collection = "LSSTComCam/runs/DRP/20241101_20241211/w_2024_51/DM-48233"

# not working perhaps because I am using w_2025_10 version
# bad : no ccd visit collection = "LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864"
# bad : no ccd visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_15/DM-50050'
# bad : no cce visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864'
# bad : no cce visit collection collection = 'LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751'
instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
where_clause = "instrument = '" + instrument + "'"
collectionStr = collection.replace("/", "_")
BANDSEL = "r"  # Most fields were observed in red filter

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
skymap = butler.get("skyMap", skymap=skymapName, collections=collection)

In [ ]:
camera = butler.get("camera", collections=collection, instrument=instrument)

## List of Strong Lenses
- article : https://arxiv.org/pdf/1104.0931

In [ ]:
#15422 44 03:32:38.21 –27:56:53.2 
ra1 = "03:32:38.21 hours"
dec1 = "-27:56:53.2 degrees"

#34244 94 03:32:06.45 –27:47:28.6 
ra2 = "03:32:06.45 hours"
dec2 = "-27:47:28.6 degrees"

#40173 35 03:33:19.45 –27:44:50.0 
ra3 = "03:33:19.45 hours"
dec3 = "-27:44:50.0 degrees"

#43242 45 03:31:55.35 –27:43:23.5 
ra4 = "03:31:55.35 hours"
dec4 = "-27:43:23.5 degrees"

#46446 47 03:31:35.94 –27:41:48.2 
ra5 = "03:31:35.94 hours"
dec5 = "-27:41:48.2 degrees"

#12589 03:31:24.89 −27:58:07.0
ra6 = "03:31:24.89 hours"
dec6 = "-27:58:07.0 degrees"

#43797 03:31:31.74 −27:43:00.8 
ra7 = "03:31:31.74 hours"
dec7 = "-27:43:00.8 degrees"

#28294 03:31:50.54 −27:50:28.4 
ra8 = "03:31:50.54 hours"
dec8 = "-27:50:28.4 degrees"

#36857 03:31:53.24 −27:46:18.9
ra9 = "03:31:53.24 hours"
dec9 = "-27:46:18.9 degrees"

#36714 03:32:59.78 −27:46:26.4 
ra10 = "03:32:59.78 hours"
dec10 = "-27:46:26.4 degrees"

In [ ]:
ra = Angle(ra10)
print(ra.degree)
dec = Angle(dec10)
print(dec.degree)

## List of Sky field of interest

In [ ]:
lsstcomcam_targets = {}
# high rank
lsstcomcam_targets["ECDFS_G15422"] = {"field_name": "GEMS-15422", "ra": 53.159208333333325, "dec": -27.94811111111111}
lsstcomcam_targets["ECDFS_G34244"] = {"field_name": "GEMS-34244", "ra": 53.02687499999999 , "dec": -27.79127777777778}
lsstcomcam_targets["ECDFS_G40173"] = {"field_name": "GEMS-40173", "ra": 53.33104166666666 , "dec": -27.747222222222224}
lsstcomcam_targets["ECDFS_G43242"] = {"field_name": "GEMS-43242", "ra": 52.980624999999996 , "dec": -27.72319444444444}
lsstcomcam_targets["ECDFS_G46446"] = {"field_name": "GEMS-46446", "ra": 52.89975 , "dec": -27.696722222222224}

# low rank
lsstcomcam_targets["ECDFS_G12589"] = {"field_name": "GEMS-12589", "ra": 52.85370833333333, "dec": -27.96861111111111}
lsstcomcam_targets["ECDFS_G43797"] = {"field_name": "GEMS-43797", "ra": 52.88224999999999, "dec": -27.71688888888889}

lsstcomcam_targets["ECDFS_G28294"] = {"field_name": "GEMS-28294", "ra": 52.960583333333325 , "dec": -27.84122222222222}
lsstcomcam_targets["ECDFS_G6857"] = {"field_name": "GEMS-6857", "ra": 52.97183333333333 , "dec": -27.771916666666666}
lsstcomcam_targets["ECDFS_G36714"] = {"field_name": "GEMS-36714", "ra": 53.249083333333324, "dec": -27.773999999999997}


In [ ]:
df = pd.DataFrame(lsstcomcam_targets).T

In [ ]:
df

### Select the target

In [ ]:
key = "ECDFS_G15422"
#key = "ECDFS_G34244"
#key = "ECDFS_G40173"
#key= "ECDFS_G43242"
#key= "ECDFS_G46446"
#key = "ECDFS_G12589"
#key = "ECDFS_G43797"
#key = "ECDFS_G28294"
#key = "ECDFS_G6857"
#key = "ECDFS_G36714"

the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_name = the_target["field_name"]

target_title = (
    the_target["field_name"] + f" band  {BANDSEL} " + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)
target_point = SpherePoint(target_ra, target_dec, degrees)

## Find the list of tract numbers from Object Table

In [ ]:
datasettype = "objectTable_tract"
therefs = butler.registry.queryDatasets(datasettype, collections=collection)
tractsId_list = np.unique([ref.dataId["tract"] for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

## Search all deepCoadd

- deepCoadd_calexp comes with WCS

In [ ]:
# List all  deepCoadd_calexp which are in the butler collection
# Thus all patch and tracts
# refs = butler.registry.queryDatasets("deepCoadd_calexp", collections = collection)
# for ref in refs:
#    print(ref.dataId)

## Find the DataId

In [ ]:
tract_info = skymap.findTract(target_point)
patch_info = tract_info.findPatch(target_point)
bbox = patch_info.getOuterBBox()

print("Patch bounding box:", bbox)

print("Tract ID :", tract_info.getId())
tractNbSel = tract_info.getId()

print("Patch Index :", patch_info.getIndex(), " , ", patch_info.getSequentialIndex())  # (x, y)
print("Bounding Box", bbox)

patchNbSel = patch_info.getSequentialIndex()

In [ ]:
central_patch = patch_info.getIndex()
central_x, central_y = patch_info.getIndex()
neighbor_patches = [
    f"{x},{y}"
    for x in range(central_x - 1, central_x + 2)
    for y in range(central_y - 1, central_y + 2)
    if 0 <= x <= 8 and 0 <= y <= 8
]

In [ ]:
neighbor_patches

In [ ]:
neighbor_patches_indexes = [
    Index2D(x=x, y=y)
    for x in range(central_x - 1, central_x + 2)
    for y in range(central_y - 1, central_y + 2)
    if 0 <= x <= 8 and 0 <= y <= 8
]

In [ ]:
neighbor_patches_seqindexes = [
    tract_info[patch_index].getSequentialIndex() for patch_index in neighbor_patches_indexes
]

In [ ]:
mapdict_patchesindexes = {}
for patch_index in neighbor_patches_indexes:
    patch_seqindex = tract_info[patch_index].getSequentialIndex()
    mapdict_patchesindexes[patch_seqindex] = f"{patch_index.x},{patch_index.y}"
mapdict_patchesindexes

In [ ]:
# Add the patch and band to the dataId, we didn't need them for the objectTable_tract because it covers all patches and bands
# However the coadds are stored by patch and band dimensions so we have to add them to the dataId

dataId = {"band": BANDSEL, "tract": tractNbSel, "patch": patchNbSel, "skymap": skymapName}

In [ ]:
full_target_title = target_title + f"(t,p) = ({tractNbSel}, {patchNbSel})"

## Fetch the DeepCoadd

In [ ]:
coadd_exp = butler.get("deepCoadd_calexp", dataId)

## Plot in matplotlib

In [ ]:
image_array = coadd_exp.image.array
image = coadd_exp.image
wcs = coadd_exp.getWcs()
psf = coadd_exp.getPsf()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
im = ax.imshow(image_array, cmap="gray", origin="lower", vmin=0, vmax=2000)
ax.set_title(full_target_title)
plt.colorbar(im, ax=ax)
plt.show()

In [ ]:
from astropy.wcs import WCS
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval

# Get astropy WCS to plot accordingly
wcs_astropy = WCS(wcs.getFitsMetadata())  # Alternative en extrayant l'entête FITS

# Use zscale to norm
interval = ZScaleInterval()
vmin, vmax = interval.get_limits(image_array)


fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1, projection=wcs_astropy)
im = ax.imshow(image_array, origin="lower", cmap="gray", vmin=vmin, vmax=vmax)

ax.set_xlabel("RA (deg)")
ax.set_ylabel("Dec (deg)")
ax.coords.grid(True, color="white", ls="dotted")
plt.title("DeepCoadd_calexp for " + target_title)
# plt.colorbar(im, ax=ax)

for index, row in df.iterrows():
            ra_deg = row["ra"]
            dec_deg = row["dec"]
            tname = row["field_name"]

            if tname == target_name: 
                ax.plot(ra_deg, dec_deg, marker='+', transform=ax.get_transform('icrs') ,color='red', markersize=20, label="target_name")

ax.legend()
plt.show()

- don't understand why plotting not working :

- https://docs.astropy.org/en/stable/visualization/wcsaxes/overlays.html

## Plot with Firefly

In [ ]:
afwDisplay.setDefaultBackend("firefly")
# display = afwDisplay.Display(frame=1)
# display.scale("asinh", "zscale")
# display.mtv(image, title = target_title)

In [ ]:

all_dataIds = [
    {
        "band": band,
        "tract": tract_info.getId(),
        "patch": patch_info.getSequentialIndex(),
        "skymap": skymapName,
    }
    for band in all_bands
]

In [ ]:
all_dataIds

In [ ]:
all_deepCoadds = []
all_deepCoadds_wcs = []
all_titles = []
for idx, band in enumerate(all_bands):
    the_band = band
    the_dataId = all_dataIds[idx]
    the_title = key + f" band {the_band}"
    try:
        coadd_exp = butler.get("deepCoadd_calexp", the_dataId)
        wcs = coadd_exp.getWcs()
        psf = coadd_exp.getPsf()
        all_deepCoadds.append(coadd_exp)
        all_deepCoadds_wcs.append(wcs)
        all_titles.append(the_title)

    except Exception as inst:
        print(f"{key} :: catch Exception for band {band}")
        print(type(inst))  # the exception type
        print(inst.args)  # arguments stored in .args
        print(inst)  # __str_

In [ ]:
N = len(all_deepCoadds)
for count in range(N):
    band = all_bands[count]
    display = afwDisplay.Display(frame=count + 1)
    # cannot succeed to show white stars on dark sky
    display.setImageColormap("gray")
    display.scale("asinh", "zscale")
    display.mtv(all_deepCoadds[count].image, title=all_titles[count])

    # work on targets
    wcs = all_deepCoadds_wcs[count]
    
    x_name = "ra"
    y_name = "dec"
    f_name = "field_name"

    with display.Buffering():
        for index, row in df.iterrows():

            ra_deg = row[x_name]
            dec_deg = row[y_name]
            tname = row[f_name]

            if tname == target_name: 
                sky_coord = SpherePoint(ra_deg * degrees, dec_deg * degrees) 
                pixel_coord = wcs.skyToPixel(sky_coord)

                xpix = pixel_coord.x
                ypix = pixel_coord.y
            
                display.dot("o", xpix, ypix, size=50, ctype=all_bands_colors[count])
                display.dot("+", xpix, ypix, size=100, ctype=all_bands_colors[count])
           

In [ ]:
# display.clearViewer()

In [ ]:
# setImageColormap) are “gray” and “grey”